# Seleção de Jazidas
A partir dos parâmetros desejados para um determinado aterro, este código seleciona a opção mais econômica de aterro com o material das jazidas disponibilizadas

In [30]:
#Dados de entrada - Aterro

import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Dados do aterro (inseridos diretamente no código)
aterro = {
    'vta': 53000,  # Volume total (Vt) em m³ do aterro
    'w': 15,       # Umidade (w) em %
    'ea': 0.52, # Índice de vazios (e) do aterro
    'gama_d': None,        # Peso específico seco (γd) em kN/m³
    'gama_s': 26,        # Peso específico dos sólidos (γs) em kN/m³
    'S': 75,     # Grau de saturação (S) em %
    'g_s': 2.6,           # Peso específico dos grãos (Gs)
    'vsa': 34868.42 # Volume de sólidos necessário (Vs) em m³ para o aterro
}

# Função para carregar dados das jazidas de um arquivo Excel
def carregar_jazidas(caminho_arquivo):
    try:
        jazidas = pd.read_excel(caminho_arquivo)
        print("Dados das jazidas carregados com sucesso.")
        return jazidas
    except Exception as e:
        print(f"Erro ao carregar arquivo: {e}")
        return None

# Função para plotar a tabela de jazidas
def plotar_jazidas(jazidas):
    if jazidas is None:
        print("Nenhuma jazida disponível para plotar.")
        return
    
    fig, ax = plt.subplots(figsize=(10, 4))  # Define o tamanho da figura
    ax.xaxis.set_visible(False)  # Esconde os eixos
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)  # Remove a borda ao redor da tabela
    tabela = table(ax, jazidas, loc='center', colWidths=[0.1]*len(jazidas.columns))  # Cria a tabela
    tabela.auto_set_font_size(False)
    tabela.set_fontsize(10)
    tabela.scale(1.2, 1.2)  # Ajusta o tamanho da tabela
    plt.title("Tabela de Jazidas")
    plt.show()

# Função para plotar os dados do aterro
def plotar_dados_aterro(aterro):
    fig, ax = plt.subplots(figsize=(6, 2))  # Define o tamanho da figura
    ax.xaxis.set_visible(False)  # Esconde os eixos
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)  # Remove a borda ao redor da tabela
    
    # Converte o dicionário do aterro em DataFrame para plotagem
    aterro_df = pd.DataFrame(aterro.items(), columns=['Parâmetro', 'Valor'])
    
    tabela = table(ax, aterro_df, loc='center', colWidths=[0.2, 0.2])  # Cria a tabela
    tabela.auto_set_font_size(False)
    tabela.set_fontsize(10)
    tabela.scale(1.2, 1.2)  # Ajusta o tamanho da tabela
    plt.title("Dados de Entrada do Aterro")
    plt.show()

# Exemplo de uso da função para carregar as jazidas
caminho_planilha = "C:\\Users\\Dell\\OneDrive\\Documentos\\GitHub\\Polatm_RamonVieira\\inputs\\DadosJazidas.xlsx"
jazidas = carregar_jazidas(caminho_planilha)

# Exibir os dados das jazidas carregados
if jazidas is not None:
    print(jazidas.head())

# Exibir os dados do aterro
if aterro is not None:
    print(aterro)

Dados das jazidas carregados com sucesso.
   jazida  custo    ej    vtj
0       1   7.65  0.91  22500
1       2   8.35  0.87  16900
2       3   9.85  0.75  27450
3       4  11.30  0.69  16800
4       5  12.90  0.67  14600
{'vta': 53000, 'w': 15, 'ea': 0.52, 'gama_d': None, 'gama_s': 26, 'S': 75, 'g_s': 2.6, 'vsa': 34868.42}


In [34]:
# Função para carregar dados das jazidas de um arquivo Excel
def carregar_jazidas(caminho_arquivo):
    try:
        jazidas = pd.read_excel(caminho_arquivo)
        print("Dados das jazidas carregados com sucesso.")
        return jazidas
    except Exception as e:
        print(f"Erro ao carregar arquivo: {e}")
        return None

# Função para calcular o volume de sólidos disponíveis
def calcular_volume_solidos(jazidas):
    if 'vtj' not in jazidas.columns or 'ej' not in jazidas.columns:
        print("Erro: Colunas 'vtj' ou 'ej' não encontradas na planilha de jazidas.")
        return None

    # Adiciona nova coluna "Volume de sólidos disponíveis (m³)"
    jazidas['Volume de sólidos disponíveis (m³)'] = jazidas['vtj'] / (1 + jazidas['ej'])
    
    print("Volume de sólidos disponíveis calculado com sucesso.")
    return jazidas

# Função para calcular o Volume Total Necessário (m³)
def calcular_volume_total_necessario(jazidas, volume_solidos_aterro):
    if 'ej' not in jazidas.columns:
        print("Erro: Coluna 'ej' não encontrada na planilha de jazidas.")
        return None

    # Adiciona nova coluna "Volume Total Necessário (m³)"
    jazidas['Volume Total Necessário (m³)'] = aterro['vsa'] * (1 + jazidas['ej'])
    
    print("Volume total necessário calculado com sucesso.")
    return jazidas

# Função para calcular o custo total necessário (R$)
def calcular_custo_total(jazidas):
    if 'Volume Total Necessário (m³)' not in jazidas.columns or 'custo' not in jazidas.columns:
        print("Erro: Colunas 'Volume Total Necessário (m³)' ou 'custo' não encontradas na planilha de jazidas.")
        return None
    
    # Adiciona uma nova coluna para o custo total (R$)
    jazidas['Custo Total (R$)'] = jazidas['Volume Total Necessário (m³)'] * jazidas['custo']
    
    print("Custo total calculado com sucesso.")
    return jazidas

# Função para selecionar o volume de sólidos do material baseado no menor custo
def selecionar_volume_solidos(jazidas, volume_solidos_necessario):
    # Ordenar as jazidas pela coluna 'custo' em ordem crescente
    jazidas = jazidas.sort_values(by='custo').reset_index(drop=True)
    
    # Inicializa o volume restante com o volume de sólidos necessário
    volume_restante = volume_solidos_necessario
    
    # Criar uma nova coluna para armazenar o Volume de sólidos do material selecionado (m³)
    jazidas['Volume de sólidos do material selecionado (m³)'] = 0.0  # Inicialmente como float
    
    # Iterar pelas jazidas
    for i, row in jazidas.iterrows():
        volume_disponivel = row['Volume de sólidos disponíveis (m³)']
        
        if volume_restante <= 0:
            break
        
        # Se o volume disponível é maior ou igual ao volume restante
        if volume_disponivel >= volume_restante:
            jazidas.at[i, 'Volume de sólidos do material selecionado (m³)'] = volume_restante
            volume_restante = 0  # O volume necessário foi preenchido, então paramos o processo
        else:
            # Se o volume disponível é menor que o volume restante
            jazidas.at[i, 'Volume de sólidos do material selecionado (m³)'] = volume_disponivel
            volume_restante -= volume_disponivel  # Subtraímos o volume disponível do restante
    
    print("Seleção do volume de sólidos do material completada.")
    return jazidas

# Função para calcular o Volume total da jazida selecionada
def calcular_volume_total_jazida_selecionada(jazidas):
    if 'Volume de sólidos do material selecionado (m³)' not in jazidas.columns or 'ej' not in jazidas.columns:
        print("Erro: Colunas 'Volume de sólidos do material selecionado (m³)' ou 'ej' não encontradas na planilha de jazidas.")
        return None
    
    # Adiciona uma nova coluna para o Volume total da jazida selecionada (m³)
    jazidas['Volume total da jazida selecionada (m³)'] = jazidas['Volume de sólidos do material selecionado (m³)'] * (1 + jazidas['ej'])
    
    print("Volume total da jazida selecionada calculado com sucesso.")
    return jazidas

# Função para calcular o custo total da jazida selecionada
def calcular_custo_total_jazida_selecionada(jazidas):
    if 'Volume total da jazida selecionada (m³)' not in jazidas.columns or 'custo' not in jazidas.columns:
        print("Erro: Colunas 'Volume total da jazida selecionada (m³)' ou 'custo' não encontradas.")
        return None
    
    # Adiciona uma nova coluna para o custo total da jazida selecionada (R$)
    jazidas['Custo Total da Jazida Selecionada (R$)'] = jazidas['Volume total da jazida selecionada (m³)'] * jazidas['custo']
    
    print("Custo total da jazida selecionada calculado com sucesso.")
    return jazidas

# Caminho do arquivo de dados das jazidas
caminho_planilha = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\DadosJazidas.xlsx"

# Carregar os dados das jazidas
jazidas = carregar_jazidas(caminho_planilha)

# Se os dados foram carregados com sucesso, calcula o volume de sólidos
if jazidas is not None:
    jazidas = calcular_volume_solidos(jazidas)
    
    # Calcular o Volume Total Necessário usando o volume de sólidos do aterro
    jazidas = calcular_volume_total_necessario(jazidas, aterro['vsa'])

    # Calcular o custo total com base no volume total necessário
    jazidas = calcular_custo_total(jazidas)

    # Selecionar o volume de sólidos do material baseado no menor custo
    jazidas = selecionar_volume_solidos(jazidas, aterro['vsa'])

    # Calcular o Volume total da jazida selecionada
    jazidas = calcular_volume_total_jazida_selecionada(jazidas)

    # Calcular o Custo total da jazida selecionada
    jazidas = calcular_custo_total_jazida_selecionada(jazidas)

    # Exibir os dados da tabela atualizados com os volumes e custos selecionados
    print(jazidas[['custo', 'Volume de sólidos do material selecionado (m³)', 'Volume total da jazida selecionada (m³)', 'Custo Total da Jazida Selecionada (R$)']])

Dados das jazidas carregados com sucesso.
Volume de sólidos disponíveis calculado com sucesso.
Volume total necessário calculado com sucesso.
Custo total calculado com sucesso.
Seleção do volume de sólidos do material completada.
Volume total da jazida selecionada calculado com sucesso.
Custo total da jazida selecionada calculado com sucesso.
   custo  Volume de sólidos do material selecionado (m³)  \
0   7.65                                    11780.104712   
1   8.35                                     9037.433155   
2   9.35                                     9994.382022   
3   9.85                                     4056.500110   
4  11.30                                        0.000000   
5  12.90                                        0.000000   

   Volume total da jazida selecionada (m³)  \
0                             22500.000000   
1                             16900.000000   
2                             17790.000000   
3                              7098.875193   
4   

In [37]:
# Função para exibir a lista das jazidas selecionadas e os totais
def exibir_lista_jazidas_selecionadas(jazidas):
    # Filtrar as jazidas selecionadas
    jazidas_selecionadas = jazidas[jazidas['Volume de sólidos do material selecionado (m³)'] > 0]
    
    # Exibir a lista das jazidas selecionadas, volume e custo
    print("\nLista das jazidas selecionadas:")
    for i, row in jazidas_selecionadas.iterrows():
        print(f"Jazida {i+1}:")
        print(f"  Volume Total (m³): {row['Volume total da jazida selecionada (m³)']:.2f}")
        print(f"  Custo Total (R$): {row['Custo Total da Jazida Selecionada (R$)']:.2f}")
    
    # Calcular a soma do volume total e do custo total
    volume_total = jazidas_selecionadas['Volume total da jazida selecionada (m³)'].sum()
    custo_total = jazidas_selecionadas['Custo Total da Jazida Selecionada (R$)'].sum()
    
    # Exibir os totais
    print(f"\nSoma do Volume Total: {volume_total:.2f} m³")
    print(f"Soma do Custo Total: R$ {custo_total:.2f}")

# Caminho do arquivo de dados das jazidas
caminho_planilha = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\DadosJazidas.xlsx"

# Carregar os dados das jazidas
jazidas = carregar_jazidas(caminho_planilha)

# Se os dados foram carregados com sucesso, calcula o volume de sólidos
if jazidas is not None:
    jazidas = calcular_volume_solidos(jazidas)
    jazidas = calcular_volume_total_necessario(jazidas, aterro['vsa'])
    jazidas = calcular_custo_total(jazidas)
    
    # Selecionar o volume de sólidos das jazidas pelo menor custo
    jazidas = selecionar_volume_solidos(jazidas, aterro['vsa'])
    
    # Calcular o volume total e o custo das jazidas selecionadas
    jazidas = calcular_volume_total_jazida_selecionada(jazidas)
    jazidas = calcular_custo_total_jazida_selecionada(jazidas)

    # Exibir a lista das jazidas selecionadas
    exibir_lista_jazidas_selecionadas(jazidas)

Dados das jazidas carregados com sucesso.
Volume de sólidos disponíveis calculado com sucesso.
Volume total necessário calculado com sucesso.
Custo total calculado com sucesso.
Seleção do volume de sólidos do material completada.
Volume total da jazida selecionada calculado com sucesso.
Custo total da jazida selecionada calculado com sucesso.

Lista das jazidas selecionadas:
Jazida 1:
  Volume Total (m³): 22500.00
  Custo Total (R$): 172125.00
Jazida 2:
  Volume Total (m³): 16900.00
  Custo Total (R$): 141115.00
Jazida 3:
  Volume Total (m³): 17790.00
  Custo Total (R$): 166336.50
Jazida 4:
  Volume Total (m³): 7098.88
  Custo Total (R$): 69923.92

Soma do Volume Total: 64288.88 m³
Soma do Custo Total: R$ 549500.42


In [39]:
import pandas as pd

# Função para salvar os resultados em um arquivo Excel
def salvar_resultados_excel(jazidas, caminho_dados_originais, caminho_saida):
    # Carregar a planilha original
    dados_originais = pd.read_excel(caminho_dados_originais)
    
    # Filtrar apenas as jazidas selecionadas
    jazidas_selecionadas = jazidas[jazidas['Volume de sólidos do material selecionado (m³)'] > 0]
    
    # Criar um ExcelWriter para salvar os dados em múltiplas abas
    with pd.ExcelWriter(caminho_saida, engine='xlsxwriter') as writer:
        # Escrever os dados originais em uma aba
        dados_originais.to_excel(writer, sheet_name='Dados Originais', index=False)
        
        # Escrever os resultados das jazidas selecionadas em outra aba
        jazidas_selecionadas.to_excel(writer, sheet_name='Jazidas Selecionadas', index=False)
        
        # Criar um resumo com os totais
        resumo = {
            'Soma do Volume Total (m³)': [jazidas_selecionadas['Volume total da jazida selecionada (m³)'].sum()],
            'Soma do Custo Total (R$)': [jazidas_selecionadas['Custo Total da Jazida Selecionada (R$)'].sum()]
        }
        
        # Escrever o resumo em outra aba
        pd.DataFrame(resumo).to_excel(writer, sheet_name='Resumo', index=False)

# Função para exibir a lista das jazidas selecionadas e os totais
def exibir_lista_jazidas_selecionadas(jazidas):
    # Filtrar as jazidas selecionadas
    jazidas_selecionadas = jazidas[jazidas['Volume de sólidos do material selecionado (m³)'] > 0]
    
    # Exibir a lista das jazidas selecionadas, volume e custo
    print("\nLista das jazidas selecionadas:")
    for i, row in jazidas_selecionadas.iterrows():
        print(f"Jazida {i+1}:")
        print(f"  Volume Total (m³): {row['Volume total da jazida selecionada (m³)']:.2f}")
        print(f"  Custo Total (R$): {row['Custo Total da Jazida Selecionada (R$)']:.2f}")
    
    # Calcular a soma do volume total e do custo total
    volume_total = jazidas_selecionadas['Volume total da jazida selecionada (m³)'].sum()
    custo_total = jazidas_selecionadas['Custo Total da Jazida Selecionada (R$)'].sum()
    
    # Exibir os totais
    print(f"\nSoma do Volume Total: {volume_total:.2f} m³")
    print(f"Soma do Custo Total: R$ {custo_total:.2f}")

# Caminho do arquivo de dados das jazidas
caminho_planilha = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\DadosJazidas.xlsx"
# Caminho de saída do novo arquivo Excel
caminho_saida_excel = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\outputs\ResultadosJazidas.xlsx"

# Carregar os dados das jazidas
jazidas = carregar_jazidas(caminho_planilha)

# Se os dados foram carregados com sucesso, calcula o volume de sólidos
if jazidas is not None:
    jazidas = calcular_volume_solidos(jazidas)
    jazidas = calcular_volume_total_necessario(jazidas, aterro['vsa'])
    jazidas = calcular_custo_total(jazidas)
    
    # Selecionar o volume de sólidos das jazidas pelo menor custo
    jazidas = selecionar_volume_solidos(jazidas, aterro['vsa'])
    
    # Calcular o volume total e o custo das jazidas selecionadas
    jazidas = calcular_volume_total_jazida_selecionada(jazidas)
    jazidas = calcular_custo_total_jazida_selecionada(jazidas)

    # Exibir a lista das jazidas selecionadas
    exibir_lista_jazidas_selecionadas(jazidas)
    
    # Salvar os resultados em um arquivo Excel
    salvar_resultados_excel(jazidas, caminho_planilha, caminho_saida_excel)

Dados das jazidas carregados com sucesso.
Volume de sólidos disponíveis calculado com sucesso.
Volume total necessário calculado com sucesso.
Custo total calculado com sucesso.
Seleção do volume de sólidos do material completada.
Volume total da jazida selecionada calculado com sucesso.
Custo total da jazida selecionada calculado com sucesso.

Lista das jazidas selecionadas:
Jazida 1:
  Volume Total (m³): 22500.00
  Custo Total (R$): 172125.00
Jazida 2:
  Volume Total (m³): 16900.00
  Custo Total (R$): 141115.00
Jazida 3:
  Volume Total (m³): 17790.00
  Custo Total (R$): 166336.50
Jazida 4:
  Volume Total (m³): 7098.88
  Custo Total (R$): 69923.92

Soma do Volume Total: 64288.88 m³
Soma do Custo Total: R$ 549500.42
